### Inference from a list of echo studies ###

In [1]:
import os
import glob
import pickle
import pandas as pd

# Custom imports
from werdich_cfr.tfutils.Modeltrainer import VideoTrainer

TensorFlow Version: 2.2.0


In [3]:
def read_model_dict(file):
    with open(file, 'rb') as f:
        model_dict = pickle.load(f)
    return model_dict

# Directories and data sets
cfr_data_root = os.path.normpath('/mnt/obi0/andreas/data/cfr')
meta_date = '200606'
meta_dir = os.path.join(cfr_data_root, 'metadata_'+meta_date)
file_df_file = 'echo_BWH_npy_feather_files_'+meta_date+'.parquet'
view = 'a4c'

# echo list
echo_list_dir = os.path.normpath('/mnt/obi0/andreas/data/cfr/predictions_echodata/SecondEchoGenetics')
echo_list_filename = 'a4cname_MGH.txt'
echo_list_file = os.path.join(echo_list_dir, echo_list_filename)
echo_list = pd.read_csv(echo_list_file, header='infer', sep='\t')
echo_file_list = list(echo_list.names.unique())
print(f'Original file list: {len(echo_file_list)} files.')

best_models = pd.read_parquet(os.path.join(echo_list_dir, 'cfr_models_200611.parquet')).reset_index(drop=True)

Original file list: 13273 files.


In [4]:
echo_list.head().names.iloc[0]

'4f2101e60be2be_4903a44ab09602c1f5c8e634489d_Image-45.npy.lz4'

In [3]:
# Create a reduced npy_feather files to speed up the metadata collections

#file_df = pd.read_parquet(os.path.join(meta_dir, file_df_file))
#file_df_study = file_df[file_df.study.isin(study_list)]
#file_df_study_list = list(file_df_study.study.unique())
#print(len(study_list))
#print(len(file_df_study_list))
# What's missing?
#missing_files = list(set(study_list).difference(file_df_study_list))
#print(len(missing_files))

# Save missing echo list
#echo_list_file_missing = echo_list_filename.split('.')[0]+'_missing.parquet'
#echo_list_missing = echo_list[echo_list.study.isin(missing_files)]
#echo_list_missing.to_parquet(os.path.join(meta_dir, echo_list_file_missing))

# Save the new list to collect the metadata
#file_df_file_pred = os.path.join(meta_dir, 'echo_BWH_npy_feather_files_pred_'+meta_date+'.parquet')
#file_df_study.to_parquet(file_df_file_pred)

7860
7705
155


In [5]:
# Join the echo list with the meta data and filter VIEWS!!!
meta_file = os.path.join(cfr_data_root, 'metadata_200606', 'echo_BWH_meta_200606.parquet')
meta_df = pd.read_parquet(meta_file)
meta_df.head(2)

,filename,dir,study,mrn,datetime,fileid,institution,model,manufacturer,index,frame_time,number_of_frames,heart_rate,deltaX,deltaY,a2c,a2c_laocc,a2c_lvocc_s,a3c,a3c_laocc,a3c_lvocc_s,a4c,a4c_far,a4c_laocc,a4c_lvocc_s,a4c_rv,a4c_rv_laocc,a5c,apex,other,plax_far,plax_lac,plax_laz,plax_laz_ao,plax_plax,psax_avz,psax_az,psax_mv,psax_pap,rvinf,subcostal,suprasternal
0,48b09010a09a4b2e_4903a582ec77f16c8f7cbc7f8bf7_...,/mnt/obi0/phi/echo/npyFiles/BWH/48b0/48b09010a...,48b09010a09a4b2e_4903a582ec77f16c8f7cbc7f8bf7,35154293,2017-06-21 13:58:47,48b09010a09a4b2e_4903a582ec77f16c8f7cbc7f8bf7_...,BWH,iE33,Philips Medical Systems,0.0,63.756,46.0,64.0,0.037484,0.037484,1.825323e-12,9.551256e-10,3.902316e-12,1.877408e-12,1.445331e-10,9.478744e-12,8.755016e-05,7.889080e-11,2.352379e-15,2.875156e-11,1.463021e-10,2.815503e-12,1.344274e-09,7.833037e-14,0.999913,3.436280e-13,3.458812e-11,4.586925e-13,1.264374e-12,7.937974e-10,1.614885e-15,3.353979e-09,2.809696e-10,1.067778e-09,4.724536e-14,4.397777e-10,2.398687e-12
1,48b09010a09a4b2e_4903a582ec77f16c8f7cbc7f8bf7_...,/mnt/obi0/phi/echo/npyFiles/BWH/48b0/48b09010a...,48b09010a09a4b2e_4903a582ec77f16c8f7cbc7f8bf7,35154293,2017-06-21 13:58:47,48b09010a09a4b2e_4903a582ec77f16c8f7cbc7f8bf7_...,BWH,iE33,Philips Medical Systems,0.0,44.333,64.0,67.0,0.041844,0.041844,1.841953e-11,5.083539e-10,1.585738e-11,1.233755e-10,1.995799e-10,1.127267e-12,1.235251e-12,4.256902e-11,9.418960e-16,7.657971e-12,1.053997e-11,6.121281e-11,5.981390e-10,2.671665e-12,1.000000,4.888175e-14,2.039909e-12,1.602643e-13,4.351348e-11,4.508235e-10,1.274972e-14,1.103503e-11,2.001570e-10,7.542065e-08,9.852938e-14,8.525817e-11,4.462386e-11


In [8]:
# Filter the meta data by the echos we need
meta_df = meta_df[meta_df.filename.isin(echo_file_list)]
print(f'Number of files in meta data after echo filter: {len(meta_df.filename.unique())}')
print(meta_df.shape)

# Remove meta rows without view classification
meta_df = meta_df.loc[~meta_df.a4c.isnull()]

# Remove rows without frame_time
meta_df = meta_df.loc[~meta_df.frame_time.isnull()].reset_index(drop = True)

print()
print('After removal of rows without view clasification:')
print('Total number of patients in meta data {}'.format(len(meta_df.mrn.unique())))
print('Total number of studies {}'.format(len(meta_df.study.unique())))
print('Total number of files in meta data {}'.format(len(meta_df.filename.unique())))

Number of files in meta data after echo filter: 0
(0, 42)

After removal of rows without view clasification:
Total number of patients in meta data 0
Total number of studies 0
Total number of files in meta data 0


In [15]:
view_dict = {'view_a2c': ['a2c', 'a2c_laocc', 'a2c_lvocc_s'],
             'view_a3c': ['a3c', 'a3c_laocc', 'a3c_lvocc_s'],
             'view_a4c': ['a4c', 'a4c_far', 'a4c_laocc', 'a4c_lvocc_s', 'a4c_rv', 'a4c_rv_laocc'],
             'view_plax': ['plax_far', 'plax_lac', 'plax_laz', 'plax_laz_ao', 'plax_plax'],
             'view_psax': ['psax_avz', 'psax_az', 'psax_mv', 'psax_pap'],
             'view_other': ['other', 'a5c', 'apex', 'rvinf', 'subcostal', 'suprasternal']}

# Get the maxiumum view classification score for each row
view_list = [item for sublist in view_dict.values() for item in sublist]
echo_meta_sum_views = meta_df.assign(max_view = meta_df[view_list].\
                                     idxmax(axis = 1))

# Make sure that we have all views in our list and for consistency: Sum up all view columns
echo_meta_sum_views = echo_meta_sum_views.assign(sum_views = echo_meta_sum_views[view_list].sum(axis=1))

print(f'Studies in data:     {len(echo_meta_sum_views.study.unique())}')
print(f'Total files:         {len(echo_meta_sum_views.filename.unique())}')

meta_df_a4c = echo_meta_sum_views[echo_meta_sum_views.max_view==view].reset_index(drop=True)

print()
print(f'Studies in a4c data: {len(meta_df_a4c.study.unique())}')
print(f'Total files:         {len(meta_df_a4c.filename.unique())}')
print(f'Total patients:      {len(meta_df_a4c.mrn.unique())}')

Studies in data:     9415
Total files:         26537

Studies in a4c data: 9415
Total files:         26537
Total patients:      5945


In [16]:
meta_a4c_filename = echo_list_filename.split('.')[0] + '_'+view+'.parquet'

In [17]:
meta_a4c_filename

'a4cname_BWH_a4c.parquet'

In [18]:
meta_a4c_filename = echo_list_filename.split('.')[0] + '_'+view+'.parquet'
meta_a4c_file = os.path.join(echo_list_dir, meta_a4c_filename)
meta_df_a4c.to_parquet(meta_a4c_file)
meta_df_a4c.head(2)

,filename,dir,study,mrn,datetime,fileid,institution,model,manufacturer,index,frame_time,number_of_frames,heart_rate,deltaX,deltaY,a2c,a2c_laocc,a2c_lvocc_s,a3c,a3c_laocc,a3c_lvocc_s,a4c,a4c_far,a4c_laocc,a4c_lvocc_s,a4c_rv,a4c_rv_laocc,a5c,apex,other,plax_far,plax_lac,plax_laz,plax_laz_ao,plax_plax,psax_avz,psax_az,psax_mv,psax_pap,rvinf,subcostal,suprasternal,max_view,sum_views
0,48b09010a09a4b2e_4903a582ec77f16c8f7cbc7f8bf7_...,/mnt/obi0/phi/echo/npyFiles/BWH/48b0/48b09010a...,48b09010a09a4b2e_4903a582ec77f16c8f7cbc7f8bf7,35154293,2017-06-21 13:58:47,48b09010a09a4b2e_4903a582ec77f16c8f7cbc7f8bf7_...,BWH,iE33,Philips Medical Systems,0.0,33.333,86.0,64.0,0.037481,0.037481,3.776899e-15,1.125900e-16,3.224137e-16,1.302460e-17,7.848898e-19,5.098876e-16,1.0,1.014299e-16,6.181438e-14,4.001530e-15,8.249603e-14,7.506383e-17,1.980705e-11,1.996741e-16,2.450561e-14,2.183899e-17,3.440755e-15,1.291427e-16,2.468740e-18,1.228838e-17,1.206828e-16,9.948373e-15,4.953818e-15,9.274850e-15,3.036023e-18,2.210231e-18,2.941698e-15,a4c,1.0
1,48b09010a09a4b2e_4903a582ec77f16c8f7cbc7f8bf7_...,/mnt/obi0/phi/echo/npyFiles/BWH/48b0/48b09010a...,48b09010a09a4b2e_4903a582ec77f16c8f7cbc7f8bf7,35154293,2017-06-21 13:58:47,48b09010a09a4b2e_4903a582ec77f16c8f7cbc7f8bf7_...,BWH,iE33,Philips Medical Systems,0.0,33.333,87.0,66.0,0.041843,0.041843,3.274194e-18,2.229899e-15,6.499876e-17,6.389318e-19,7.796556e-19,9.509607e-18,1.0,1.160418e-16,2.514978e-15,3.065668e-16,1.090487e-14,6.611535e-17,3.245944e-15,9.960633e-16,2.744140e-16,1.558221e-18,6.892581e-17,3.179071e-16,1.023685e-19,8.421285e-18,1.718870e-17,2.774656e-15,6.060843e-16,3.451965e-14,1.964853e-18,4.619642e-19,4.186211e-17,a4c,1.0


In [19]:
model = best_models.iloc[0]
model

model_name                       nondefect_a4c_dgx-1_stress_mbf_unaff
model_output                                         stress_mbf_unaff
epoch                                                             100
chechkpoint_file    /mnt/obi0/andreas/data/cfr/log/nondefect_a4c_d...
spear_cor                                                    0.446963
spear_p                                                   1.73464e-44
pear_cor                                                     0.440022
pear_p                                                    5.11224e-43
n_samples                                                         881
dset                                                        nondefect
view                                                              a4c
gpu                                                             dgx-1
Name: 0, dtype: object